# 11. Decorators: Modifying Function Behavior

Decorators are a powerful and widely used feature in Python. They are a way to **modify or enhance the behavior of functions or methods** without permanently altering their source code. You can think of a decorator as a "wrapper" that you can place around a function to `add functionality` before or after the main function runs.

This is extremely useful for tasks that apply to many different functions, such as `logging`, `timing`, `access control`, or `input validation`.

- A decorator is a function that takes another function as an argument and returns a new, modified function.
- The `@decorator_name` syntax is a clean and "Pythonic" shortcut for applying a decorator to a function.
- It allows for adding/removing functionality easily.

## 11.1. The Structure of a Decorator

In [ ]:
# Let's break down the execution flow.

def my_protocol_decorator(original_func): # 3. The decorator function receives our original function as a parameter.
    def wrapper_func(): # 5. The 'wrapper' function "encloses" our original function's call.
        print("Action BEFORE the core function executes (e.g., a security check).") # 6. This prints.
        original_func() # 7. The original function is executed here.
        print("Action AFTER the core function executes (e.g., logging the result).") # 8. This prints.
    return wrapper_func # 4. The decorator returns the new wrapper function.

@my_protocol_decorator # 2. We trigger the decorator function.
def core_mission_task():
    print(">>> Core mission task is running! <<<")

core_mission_task() # 1. When we call our function we get the decorator first.

"""
Execution Flow Summary:
1. We call `core_mission_task()`.
2. Python sees the `@my_protocol_decorator` and passes `core_mission_task` into it.
3. `my_protocol_decorator` receives `core_mission_task` as its `original_func` parameter.
4. `my_protocol_decorator` defines `wrapper_func` and returns it.
5. The name `core_mission_task` now points to the `wrapper_func`.
6. When `core_mission_task()` is executed, it's actually `wrapper_func()` that runs, executing the "before" code, then the original function, then the "after" code.
"""


# --- The Same Functionality Without Decorator Syntax ---
# This demonstrates that a decorator is a form of Higher-Order Function (HOF).
def manual_decorator(func):
    def wrapper():
        print("Manual 'before' action.")
        func() # Calling the original function
        print("Manual 'after' action.")
    return wrapper # Returns the new wrapper function

def some_manual_function():
    print(">>> Manual core function running! <<<")

# Applying the "decorator" manually
decorated_function = manual_decorator(some_manual_function)

# Calling the new function that has been "wrapped"
decorated_function()


# --- Applying Multiple Decorators ---
# You can stack multiple decorators. They are applied from the bottom up.
# Think of them as layers of an onion: the call goes through them in order from top to bottom,
# then the original function runs, then the execution comes back out through the layers in reverse order.
def protocol_layer_1(func): # Outer decorator
    def wrapper():
        print("Protocol 1: Engaged.") # 1st to execute
        func()
        print("Protocol 1: Disengaged.") # 6th to execute
    return wrapper

def protocol_layer_2(func): # Inner decorator
    def wrapper():
        print("  Protocol 2: Engaged.") # 2nd to execute
        func()
        print("  Protocol 2: Disengaged.") # 5th to execute
    return wrapper

@protocol_layer_1 # Applied first
@protocol_layer_2 # Applied second
def core_command():
    print("    >>> Core command executing! <<<") # 3rd to execute

core_command()
# 4th to execute is the print statement inside the core_command function.


## 11.2. Practical Use Cases: Logging, Validation, Timing

In [ ]:
# --- Use Case: LOGGING ---
# To log when a function is called and with what arguments.
# The wrapper must accept *args and **kwargs to handle any function signature.

def log_action(func):
    def wrapper(*args, **kwargs):
        print(f"Calling function: '{func.__name__}' with arguments {args} and kwargs {kwargs}")
        result = func(*args, **kwargs) # Execute the original function and store its result
        print(f"Function '{func.__name__}' finished execution.")
        return result # Return the result of the original function
    return wrapper

@log_action
def calculate_trajectory(planet_mass: float, initial_velocity: float):
    return (planet_mass * initial_velocity) / 9.81 # Example calculation

trajectory_result = calculate_trajectory(1.5e22, 2000)
print(f"Trajectory calculation result: {trajectory_result}")


# --- Use Case: VALIDATION ---
# To ensure arguments meet certain conditions before the function runs.
def validate_inputs(func):
    def wrapper(*args, **kwargs):
        if len(args) > 0 and isinstance(args[0], (int, float)) and args[0] > 0:
            return func(*args, **kwargs) # If valid, run the function
        else:
            raise ValueError("Input must be a positive number.") # Raise an error if invalid
    return wrapper # Note: corrected from wrapper() in original CZ, which was a bug

@validate_inputs
def set_reactor_output(power_level_mw: float):
    print(f"Setting reactor output to {power_level_mw} MW.")

set_reactor_output(100) # This would work
set_reactor_output(-50) # This would raise ValueError


# --- Use Case: TIMING ---
# To measure the execution time of a function.
from time import time

def measure_performance(func):
    def wrapper(*args, **kwargs):
        start_time = time() # Record time before execution
        result = func(*args, **kwargs) # Run the function
        end_time = time() # Record time after execution
        print(f"Function '{func.__name__}' took {end_time - start_time:.4f} seconds to execute.")
        return result
    return wrapper

@measure_performance
def run_complex_simulation():
    total = 0
    for i in range(10_000_000):
        total += i
    print("Simulation complete.")
    return total

run_complex_simulation()


# --- Decorators in OOP ---
# The same principles apply, but the wrapper must accept 'self'.

"""
Example File: `protocols.py`
"""
def log_method_call(func):
    def wrapper(self, *args, **kwargs): # Wrapper includes 'self' as the first argument
        print(f"Method '{func.__name__}' called on object ID {self.id}")
        return func(self, *args, **kwargs)
    return wrapper

"""
Example File: `probe_class.py`
"""
# from protocols import log_method_call

class AutonomousProbe:
    def __init__(self, probe_id):
        self.id = probe_id

    @log_method_call
    def send_status(self):
        print(f"Probe {self.id} reporting: All systems nominal.")

"""
Example File: `main.py`
"""
# from probe_class import AutonomousProbe

# --- Testing ---
probe_alpha = AutonomousProbe("XP-01")
probe_alpha.send_status()
# Output would show the log message from the decorator first, then the message from the method.

## practice

**Task: Create a Performance Logging Decorator**
- **Scenario:** You need a standard way to log the start, end, and execution time of various critical functions in your mission control software.
- **Requirements:**
    - Create any simple function(s) to simulate a task / work.
    - Create a decorator named `log_to_file`.
    - This decorator should perform the following actions and write them to a file named `execution.log`:
        1.  **Before** calling the wrapped function, it should write a line to the file, e.g., `"--- Function 'process_data' started at <timestamp> ---"`.
        2.  **After** the function completes, it should write another line to the file, e.g., `"--- Function 'process_data' finished at <timestamp>. Duration: X.XXXX seconds ---"`.
- **Testing:**
    - Apply your `@log_to_file` decorator to your function(s).
    - Call the decorated function(s).
    - Check the contents of `execution.log` to verify that both the start and end log entries were written correctly, including the timestamps and calculated duration.

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom